In [1]:
pip install requests beautifulsoup4 pandas openpyxl

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
     --------------------------------------- 11.6/11.6 MB 11.1 MB/s eta 0:00:00
     -------------------------------------- 250.9/250.9 kB 5.1 MB/s eta 0:00:00
     -------------------------------------- 101.8/101.8 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 70.4/70.4 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 126.3/126.3 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 167.3/167.3 kB 5.1 MB/s eta 0:00:00
  Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
     ---------------------------------------- 12.9/12.9 MB 8.4 MB/s eta 0:00:00
     ------------------------------------- 508.0/508.0 kB 10.6 MB/s eta 0:00:00
     -------------------------------------- 346.6/346.6 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# This is to create an excel file.
"""
import pandas as pd

data = {
    "Website_URLs": [
        "https://www.example.com",
        "https://www.wikipedia.org",
        "https://www.python.org"
    ]
}

excel_file_path = "website_urls.xlsx"
df = pd.DataFrame(data)
df.to_excel(excel_file_path, index=False)
print(f"Excel file with URLs created at: {excel_file_path}")
"""

'\nimport pandas as pd\n\ndata = {\n    "Website_URLs": [\n        "https://www.example.com",\n        "https://www.wikipedia.org",\n        "https://www.python.org"\n    ]\n}\n\nexcel_file_path = "website_urls.xlsx"\ndf = pd.DataFrame(data)\ndf.to_excel(excel_file_path, index=False)\nprint(f"Excel file with URLs created at: {excel_file_path}")\n'

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to fetch and clean data
def fetch_and_clean_data(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, "html.parser")
        cleaned_text = soup.get_text(separator=" ", strip=True)
        return cleaned_text
    except Exception as e:
        return f"Error fetching URL {url}: {e}"

# Read URLs from the Excel file
input_file = "website_urls.xlsx"
df = pd.read_excel(input_file)

# Process each URL
cleaned_data = []
for url in df["Website_URLs"]:
    cleaned_data.append(fetch_and_clean_data(url))

# Save the cleaned data to a new Excel file
df["Cleaned_Text"] = cleaned_data
output_file = "cleaned_website_data.xlsx"
df.to_excel(output_file, index=False)
print(f"Cleaned data saved at: {output_file}")


Cleaned data saved at: cleaned_website_data.xlsx


In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import re

# Load the Excel file
file_path = "cleaned_website_data.xlsx"  # Replace with your actual file path
df = pd.read_excel(file_path)

In [9]:
# Define the preprocessing function
def preprocess_text(raw_text):
    try:
        # Step 1: Remove HTML tags and scripts
        soup = BeautifulSoup(raw_text, "html.parser")
        cleaned_text = soup.get_text(separator=" ", strip=True)

        # Step 2: Normalize whitespace
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

        # Step 3: Remove special characters
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)

        # Step 4: Convert to lowercase
        cleaned_text = cleaned_text.lower()

        # Step 5 (Optional): Tokenization (if needed, uncomment below)
        # tokens = cleaned_text.split()

        # Step 6 (Optional): Remove stopwords (if needed, uncomment below)
        # from nltk.corpus import stopwords
        # stop_words = set(stopwords.words('english'))
        # tokens = [word for word in tokens if word not in stop_words]
        # cleaned_text = " ".join(tokens)

        return cleaned_text

    except Exception as e:
        return f"Error processing text: {e}"

In [10]:
# Apply preprocessing to the "Cleaned_Text" column
processed_df = df[['Cleaned_Text']].copy()
processed_df['Processed_Text'] = processed_df['Cleaned_Text'].apply(preprocess_text)

# Save the processed data to a separate Excel file
output_file = "processed_website_data.xlsx"
processed_df.to_excel(output_file, index=False)
print(f"Processed data saved to: {output_file}")


Processed data saved to: processed_website_data.xlsx


C:\Users\C-Dac\AppData\Local\Temp\ipykernel_10632\663602529.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(raw_text, "html.parser")


In [11]:
### 1. Generate Embeddings

In [13]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load processed data
file_path = "processed_website_data.xlsx"
df = pd.read_excel(file_path)

e:\Prabhat\NER and Attributes Extraction\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
df['Embeddings'] = df['Processed_Text'].apply(lambda text: model.encode(text).tolist())

# Save embeddings
df.to_pickle("processed_data_with_embeddings.pkl")

e:\Prabhat\NER and Attributes Extraction\myenv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\C-Dac\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [15]:
import faiss
import numpy as np

ModuleNotFoundError: No module named 'faiss'